# Extract traditional smells
returns a dictionary of list, with key represents a traditional smell and value represents list of classes that contain the smell

In [ ]:
import os
import subprocess
import configparser
import pandas as pd
import re
import shutil
import time
import sys

def extract_traditional_smells(project_name):
    print("extracting traditional smells from repository: " + project_name)
    %cd ./resources
    subprocess.call(['java', '-jar', './DECOR_JAVA.jar', "../" + project_name, project_name, ""])
    %cd ../

    # check if the result of detection is generated
    if os.path.isdir('./TEMP'):s

        # dictionary of list, key represents a traditional smell and value represents list of classes that contain the smell
        smell_classes_dict = dict()

        # iterate through each generated file
        result_files_path = "./TEMP/" + project_name
        for filename in os.listdir(result_files_path):

            # read the content of the file
            file_path = result_files_path + "/" + filename
            with open(file_path) as file:
                file_content = '[dummy_section]\n' + file.read()
            config = configparser.ConfigParser()
            config.read_string(file_content)

            # find all smelly test classes that contain traditional smell
            smelly_classes = []
            for key, value in config.items('dummy_section'):
                if re.match("^[0-9]+\.[0-9]+\.[a-zA-Z]+-\d$", key):
                    class_name = value.split(".")[-1]
                    if "test" in class_name.lower() or "tests" in class_name.lower():
                        smelly_classes.append(class_name)
            # map smell and classes containing the smell
            if smelly_classes:
                smell_name = filename.split(" ")[-1].split(".")[0]
                smell_classes_dict[smell_name] = smelly_classes

        # delete result of detection
        try:
            shutil.rmtree("./TEMP/")
        except OSError as e:
            print("Error: %s - %s." % (e.filename, e.strerror))
        finally:
            print("Traditional smells extracted for project: ", project_name)
            return smell_classes_dict

    else:    
        print("ERROR: extracting traditionial smells failed")
        return None